In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import pickle
from preprocess import load_midi_files, prepare_sequences

# LSTM Model
class MusicLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MusicLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # Last time-step
        return out

# Prepare data
notes = load_midi_files()
input_data, output_data, note_to_int, pitchnames = prepare_sequences(notes)

# Convert to Torch tensors
X = torch.tensor(input_data, dtype=torch.float32)
y = torch.tensor(output_data, dtype=torch.long)

dataset = TensorDataset(X, y)
loader = DataLoader(dataset, batch_size=64, shuffle=True)

# Model params
input_size = 1
hidden_size = 256
output_size = len(pitchnames)

model = MusicLSTM(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 50
for epoch in range(epochs):
    total_loss = 0
    for batch_X, batch_y in loader:
        optimizer.zero_grad()
        output = model(batch_X)
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(loader):.4f}")

# Save model + mappings
torch.save(model.state_dict(), "model.pth")
with open("data/mappings.pkl", "wb") as f:
    pickle.dump((note_to_int, pitchnames), f)


ModuleNotFoundError: No module named 'preprocess'